In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Particionado').master('local[5]').getOrCreate()

23/04/11 15:37:06 WARN Utils: Your hostname, PRRS-DCARVAJAL resolves to a loopback address: 127.0.1.1; using 172.30.127.46 instead (on interface eth0)
23/04/11 15:37:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/11 15:37:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.range(0, 20)
df.rdd.getNumPartitions()

5

In [7]:
rdd1 = spark.sparkContext.parallelize((0, 20), 10)
rdd1.getNumPartitions()

10

In [10]:
rdd_from_file = spark.sparkContext.textFile('files/deporte.csv', 10)

In [11]:
rdd_from_file.getNumPartitions()

10

In [12]:
rdd_from_file.saveAsTextFile('files/partition_output/')

In [13]:
!ls files/partition_output/

_SUCCESS    part-00001	part-00003  part-00005	part-00007  part-00009
part-00000  part-00002	part-00004  part-00006	part-00008


In [14]:
!head -n 5 files/partition_output/part-00001

7,Athletics
8,Ice Hockey
9,Swimming
10,Badminton
11,Sailing


In [20]:
rdd = spark.sparkContext.wholeTextFiles('files/partition_output/*')

In [17]:
rdd.take(5)

[('file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00000',
  'deporte_id,deporte\n1,Basketball\n2,Judo\n3,Football\n4,Tug-Of-War\n5,Speed Skating\n6,Cross Country Skiing\n'),
 ('file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00001',
  '7,Athletics\n8,Ice Hockey\n9,Swimming\n10,Badminton\n11,Sailing\n12,Biathlon\n13,Gymnastics\n14,Art Competitions\n'),
 ('file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00002',
  '15,Alpine Skiing\n16,Handball\n17,Weightlifting\n18,Wrestling\n19,Luge\n20,Water Polo\n'),
 ('file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00003',
  '21,Hockey\n22,Rowing\n23,Bobsleigh\n24,Fencing\n25,Equestrianism\n26,Shooting\n27,Boxing\n28,Taekwondo\n'),
 ('file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00004',
  '29,Cycling\n30,Diving\n31,Canoeing\n32,Tennis\n33,Modern Pentathlon\n34,Figure Skating\n35,Golf\n')]

In [22]:
lista = rdd.mapValues(lambda x: x.split()).collect()
lista = [i[0] for i in lista]
lista

['file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00000',
 'file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00001',
 'file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00002',
 'file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00003',
 'file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00004',
 'file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00005',
 'file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00006',
 'file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00007',
 'file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00008',
 'file:/home/daielchom/projects/spark_notebooks/files/partition_output/part-00009']

In [26]:
rdd2 = spark.sparkContext.textFile(','.join(lista), 10).map(lambda l:l.split(','))

In [27]:
rdd2.take(4)

[['deporte_id', 'deporte'],
 ['1', 'Basketball'],
 ['2', 'Judo'],
 ['3', 'Football']]